In [2]:
from elasticsearch import Elasticsearch

# Use your actual credentials here
es = Elasticsearch(
    "http://localhost:9200",
    basic_auth=("elastic", "12345678")  # ← update with your actual password
)

# Same as before
index_name = "test-index"
doc = {
    "name": "Alice2",
    "email": "alice@example.com",
    "age": 30
}

# Index the document
response = es.index(index=index_name, document=doc)
print("Document indexed:", response['_id'])

# Get the document
doc_id = response['_id']
retrieved = es.get(index=index_name, id=doc_id)
print("Retrieved document:", retrieved['_source'])


Document indexed: Lb56U5YB4PDLRd2pLXKA
Retrieved document: {'name': 'Alice2', 'email': 'alice@example.com', 'age': 30}


In [3]:
response = es.search(
    index=index_name,
    query={"match_all": {}}
)

# Print each document
print("Documents found:")
for hit in response["hits"]["hits"]:
    print(hit["_source"])

Documents found:
{'name': 'Alice', 'email': 'alice@example.com', 'age': 30}
{'name': 'Alice2', 'email': 'alice@example.com', 'age': 30}


In [22]:

from datetime import datetime, timezone
video_doc = {
    "video_id": "dQw4w9WgXcQ",
    "title": "Nevere Gonna Give You Up",
    "description": "The official music video for “Nevere Gonna Give You Up” by Rick Astley.",
    "published_at": "2009-10-25T06:57:33Z",
    "thumbnails": "https://i.ytimg.com/vi/dQw4w9WgXcQ/hqdefault.jpg",
    "indexed_at": datetime.now(timezone.utc).isoformat()
}

# Index name (make sure it exists or create it beforehand with mapping)
index_name = "youtube_videos"

# Index the document (Elasticsearch will auto-generate an ID if not provided)
response = es.index(index=index_name, document=video_doc)

print("Indexed document ID:", response['_id'])
doc_id = response['_id']
retrieved = es.get(index=index_name, id=doc_id)
print("Retrieved document:", retrieved['_source'])

Indexed document ID: L76XU5YB4PDLRd2pSXIM
Retrieved document: {'video_id': 'dQw4w9WgXcQ', 'title': 'Nevere Gonna Give You Up', 'description': 'The official music video for “Nevere Gonna Give You Up” by Rick Astley.', 'published_at': '2009-10-25T06:57:33Z', 'thumbnails': 'https://i.ytimg.com/vi/dQw4w9WgXcQ/hqdefault.jpg', 'indexed_at': '2025-04-20T14:25:45.738062+00:00'}


In [27]:
import httpx

def search_videos(query: str, score_threshold: float = 1.0):
    url = "http://localhost:8000/search"  # Replace with your actual API URL

    params = {
        "q": query
    }

    # Perform the GET request
    with httpx.Client() as client:
        try:
            response = client.get(url, params=params)

            # Raise an exception if the response status code is not 200 (OK)
            response.raise_for_status()

            # Parse the JSON response
            data = response.json()
            
            # Print or return the data
            print(data)  # For debugging
            return data

        except httpx.RequestError as exc:
            print(f"An error occurred while requesting {exc.request.url!r}.")
        except httpx.HTTPStatusError as exc:
            print(f"HTTP error occurred: {exc.response.status_code}")
        except Exception as exc:
            print(f"An error occurred: {exc}")

# Call the function with example query and score threshold
query = "gonna"
score_threshold = 1.5  # Example score threshold
search_videos(query, score_threshold)


{'total': 2, 'page': 1, 'size': 2, 'videos': [{'video_id': 'dQw4w9WgXcQ', 'title': 'Never Gonna Give You Up', 'description': 'The official music video for “Never Gonna Give You Up” by Rick Astley.', 'published_at': '2009-10-25T06:57:33Z', 'thumbnails': 'https://i.ytimg.com/vi/dQw4w9WgXcQ/hqdefault.jpg', 'indexed_at': '2025-04-20T13:53:58.263103Z'}, {'video_id': 'dQw4w9WgXcQ', 'title': 'Nevere Gonna Give You Up', 'description': 'The official music video for “Nevere Gonna Give You Up” by Rick Astley.', 'published_at': '2009-10-25T06:57:33Z', 'thumbnails': 'https://i.ytimg.com/vi/dQw4w9WgXcQ/hqdefault.jpg', 'indexed_at': '2025-04-20T14:25:45.738062Z'}]}


{'total': 2,
 'page': 1,
 'size': 2,
 'videos': [{'video_id': 'dQw4w9WgXcQ',
   'title': 'Never Gonna Give You Up',
   'description': 'The official music video for “Never Gonna Give You Up” by Rick Astley.',
   'published_at': '2009-10-25T06:57:33Z',
   'thumbnails': 'https://i.ytimg.com/vi/dQw4w9WgXcQ/hqdefault.jpg',
   'indexed_at': '2025-04-20T13:53:58.263103Z'},
  {'video_id': 'dQw4w9WgXcQ',
   'title': 'Nevere Gonna Give You Up',
   'description': 'The official music video for “Nevere Gonna Give You Up” by Rick Astley.',
   'published_at': '2009-10-25T06:57:33Z',
   'thumbnails': 'https://i.ytimg.com/vi/dQw4w9WgXcQ/hqdefault.jpg',
   'indexed_at': '2025-04-20T14:25:45.738062Z'}]}

In [13]:
import httpx

# API endpoint
url = "http://localhost:8000/videos"
params = {
    "page": 1,
    "size": 10
}

# Make the request
response = httpx.get(url, params=params)

# Handle the response
if response.status_code == 200:
    data = response.json()
    print(data)
else:
    print("Error:", response.status_code, response.text)


{'total': 1, 'page': 1, 'size': 1, 'videos': [{'video_id': 'dQw4w9WgXcQ', 'title': 'Never Gonna Give You Up', 'description': 'The official music video for “Never Gonna Give You Up” by Rick Astley.', 'published_at': '2009-10-25T06:57:33Z', 'thumbnails': 'https://i.ytimg.com/vi/dQw4w9WgXcQ/hqdefault.jpg', 'indexed_at': '2025-04-20T13:53:58.263103Z'}]}
